In [220]:
import yfinance as yf
import pandas as pd
import investpy
import numpy as np
import pandas_datareader.data as web
from dateutil.relativedelta import relativedelta
import scipy.stats as st

In [221]:
def Somers_D(xx,yy):
    if yy.isnull().sum()>0:
        return print('\\nNANS!')

    else :
        x = st.rankdata(xx)
        y = st.rankdata(yy)

        n2const = y.size*(y.size-1)/2
        n=np.zeros(np.unique(y).size)

        for i in range (0,np.unique(y).size):
            for j in range(0,y.size):
                if (y[j] == np.unique(y)[i]):
                    n[i]+=1            
        y_adj = 0

        for ii in range (0,np.unique(y).size):
            y_adj += 0.5*n[ii]*(n[ii]-1)
         
        K = 0

        for kk in range(0,y.size-1):
            for kkk in range(kk+1,y.size):
                K += np.sign(x[kk] - x[kkk])*np.sign(y[kk] - y[kkk])
        return K/(n2const-y_adj)

In [235]:
#рейтинги загружаем:
df_ordering=pd.read_excel('CTF_expert_ordering_v0.2.xlsx')
df_ordering=pd.read_excel('CTF_expert_ordering_v0.2.xlsx')
df_ordering['year']=list(map(lambda x: x.year,df_ordering.report_date))
df_ordering.head(5)

,company_name,report_date,observation_date,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,year
0,Gunvor Group,2010-01-01,2010-04-01,4.0,4.0,419,2010
1,Gunvor Group,2011-01-01,2011-04-01,4.0,4.0,424,2011
2,Gunvor Group,2012-01-01,2012-04-01,4.0,4.0,427,2012
3,Gunvor Group,2013-01-01,2013-04-01,4.0,4.0,422,2013
4,Gunvor Group,2014-01-01,2014-04-01,4.0,4.0,423,2014


In [236]:
#Информацию по индустриям добавляем:
df_industries=pd.read_excel('industries.xlsx',skiprows=1)
df_industries=df_industries[['Компания','Отрасль1','Отрасль2','Отрасль3']]
df_industries.columns=['company_name','industry1','industry2','industry3']
df_ordering = pd.merge(df_industries, df_ordering, on=["company_name"]).dropna(subset=['экспертный рейтинг (бизнес)'])

In [238]:
df_ordering.head(5)

,company_name,industry1,industry2,industry3,report_date,observation_date,экспертный рейтинг (бизнес),экспертный рейтинг (андеррайтинг),financial_score,year
3,ООО РН-Бункер,NaN,NaN,NaN,2010-01-01,2010-04-01,4.0,4.0,555,2010
5,Newcoal Trading AG,NaN,NaN,NaN,2017-01-01,2017-04-01,4.0,3.0,222,2017
6,Newcoal Trading AG,NaN,NaN,NaN,2018-01-01,2018-04-01,4.0,3.0,137,2018
7,Newcoal Trading AG,NaN,NaN,NaN,2019-01-01,2019-04-01,4.0,3.0,346,2019
8,Newcoal Trading AG,NaN,NaN,NaN,2020-01-01,2020-04-01,4.0,3.0,242,2020


In [239]:
#средняя цена нефти:
data_oil = pd.read_excel('PET_PRI_SPT_S1_A.xlsx',skiprows=2)
data_oil['year']=list(map(lambda x: x.year,data_oil['Date']))
data_oil.rename(columns={'Europe Brent Spot Price FOB (Dollars per Barrel)': 'Brent'}, inplace=True)
del data_oil['Cushing, OK WTI Spot Price FOB (Dollars per Barrel)'],data_oil['Date']
data_oil.dropna(inplace=True)
data_oil.head(5)

,Brent,year
1,18.53,1987
2,14.91,1988
3,18.23,1989
4,23.76,1990
5,20.04,1991


In [263]:
def check_hyp(df_ordering  #-это df c рейтингами компаний
             ,hyp #-это df c гипотезой)
             ):
    result = pd.merge(hyp, df_ordering, on=["year"])
    result.dropna(subset=['экспертный рейтинг (бизнес)'],inplace=True)
    df=pd.DataFrame()
    for name in result.company_name.unique():
        res=result[result['company_name']==name]
        if len(res)>1: df[name]=[Somers_D(res['экспертный рейтинг (бизнес)'], res.iloc[:,0])]
        
    return df.T.sort_values(by=0,ascending=False).rename(columns={0: hyp.columns[0]})

In [268]:
#значения Somers'-Delta:
check_hyp(df_ordering,data_oil)

,Brent
ООО Трансбункер-Ново,0.6
MMK Trading,0.0
АО Нефтебаза Красный Яр,0.0
BB Energy Trading Ltd,0.0
Far East Commodities Pte Ltd,0.0
Cetracore Energy GmbH,0.0
Newcoal Trading AG,0.0
Vitol SA,0.0
Glencore Plc,0.0
RTI LTD,0.0


In [312]:
#данные по углю загружаем:
data_coal = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "MTF=F",
        start="2011-01-01",
        end="2021-11-13",
        interval = "3mo",
        group_by = 'ticker',
    )
data_coal['Date'] = data_coal.index
data_coal['year1']=list(map(lambda x: x.year,data_coal['Date']))

[*********************100%***********************]  1 of 1 completed


In [313]:
data_coal.head(5)

,Open,High,Low,Close,Adj Close,Volume,Date,year1
Date,,,,,,,,
2010-12-01,132.000000,132.000000,119.000000,120.000000,120.000000,78,2010-12-01,2010
2011-03-01,119.750000,129.750000,119.750000,122.150002,122.150002,128,2011-03-01,2011
2011-06-01,121.849998,126.349998,120.800003,125.650002,125.650002,193,2011-06-01,2011
2011-09-01,126.199997,126.199997,111.099998,112.000000,112.000000,365,2011-09-01,2011
2011-12-01,111.949997,112.099998,97.750000,97.750000,97.750000,2135,2011-12-01,2011


In [314]:
#усредняем кварталы - получаем информацию по годам:
data_coal = data_coal.groupby('year1').agg('mean')
data_coal['year'] = data_coal.index
data_coal.columns.name 
data_coal.head(5)

,Open,High,Low,Close,Adj Close,Volume,year
year1,,,,,,,
2010,132.000000,132.000000,119.000000,120.000000,120.000000,78.000000,2010
2011,119.937498,123.599998,112.350000,114.387501,114.387501,705.250000,2011
2012,92.500000,95.312500,86.287498,89.802502,89.802502,2170.000000,2012
2013,82.924999,82.924999,76.025002,78.875000,78.875000,4362.500000,2013
2014,73.816668,75.233335,67.416667,70.300001,70.300001,7226.333333,2014


In [315]:
#тестируем:
data_coal['change']=(data_coal.Close-data_coal.Close.shift())/data_coal.Close.shift()
check_hyp(df_ordering,data_coal[['change','year']].dropna())

,change
New Stream Trading AG,0.000000
АО Чукотснаб,0.000000
BB Energy Trading Ltd,0.000000
Cetracore Energy GmbH,0.000000
Newcoal Trading AG,0.000000
Vitol SA,0.000000
Glencore Plc,0.000000
RTI LTD,0.000000
Glencore Energy UK Ltd,0.000000
АО Нефтебаза Красный Яр,0.000000


In [316]:
data_coal['change']=(data_coal.Close-data_coal.Close.shift())/data_coal.Close.shift()
check_hyp(df_ordering,data_coal[['Volume','year']].dropna())

,Volume
АО Чукотснаб,0.0
АО СПК,0.0
BB Energy Trading Ltd,0.0
Far East Commodities Pte Ltd,0.0
Cetracore Energy GmbH,0.0
Newcoal Trading AG,0.0
Vitol SA,0.0
Glencore Plc,0.0
RTI LTD,0.0
Glencore Energy UK Ltd,0.0


In [317]:
data_oil_vol = pd.read_excel('Годовая волатильность нефти.xlsx',skiprows=264).iloc[:,18:20]
data_oil_vol.columns = ['year','vol']
data_oil_vol.head(5)

,year,vol
0,2021,34.223531
1,2020,129.761544
2,2019,32.387562
3,2018,29.714897
4,2017,26.010388


In [318]:
check_hyp(df_ordering,data_oil_vol.dropna())

,year
Far East Commodities Pte Ltd,0.666667
Newcoal Trading AG,0.000000
АО Ямалгосснаб,0.000000
RTI LTD,0.000000
Alpicom,0.000000
Vitol SA,0.000000
Glencore Plc,0.000000
BB Energy Trading Ltd,0.000000
Glencore Energy UK Ltd,0.000000
SOCAR Trading SA,0.000000


In [319]:
data_oil['change']=(data_oil.Brent-data_oil.Brent.shift())/data_oil.Brent.shift()

In [321]:
data_oil.head(5)

,Brent,year,change
1,18.53,1987,NaN
2,14.91,1988,-0.195359
3,18.23,1989,0.222669
4,23.76,1990,0.303346
5,20.04,1991,-0.156566


In [322]:
check_hyp(df_ordering,data_oil[['change','year']].dropna())

,change
ООО Трансбункер-Ново,0.600000
MMK Trading,0.000000
АО Нефтебаза Красный Яр,0.000000
BB Energy Trading Ltd,0.000000
Cetracore Energy GmbH,0.000000
Newcoal Trading AG,0.000000
Vitol SA,0.000000
Glencore Plc,0.000000
RTI LTD,0.000000
Glencore Energy UK Ltd,0.000000


In [323]:
data_coal['change']=(data_coal.Close-data_coal.Close.shift())/data_coal.Close.shift()
check_hyp(df_ordering,data_coal[['change','year']].dropna())

,change
New Stream Trading AG,0.000000
АО Чукотснаб,0.000000
BB Energy Trading Ltd,0.000000
Cetracore Energy GmbH,0.000000
Newcoal Trading AG,0.000000
Vitol SA,0.000000
Glencore Plc,0.000000
RTI LTD,0.000000
Glencore Energy UK Ltd,0.000000
АО Нефтебаза Красный Яр,0.000000
